<a href="https://colab.research.google.com/github/JeLaKo/apple-tree-disease/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the data directly through Kaggle's API

In [134]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [136]:
!pip install -U -q kaggle==1.5.8

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [2]:
%cd /content/gdrive/MyDrive/MLP/

/content/gdrive/MyDrive/MLP


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/MLP/"
! kaggle competitions download -c plant-pathology-2021-fgvc8

  0% 0.00/888k [00:00<?, ?B/s]
100% 888k/888k [00:00<00:00, 29.4MB/s]
  0% 0.00/964k [00:00<?, ?B/s]
100% 964k/964k [00:00<00:00, 63.7MB/s]
  0% 0.00/991k [00:00<?, ?B/s]
100% 991k/991k [00:00<00:00, 64.4MB/s]
  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 65.0MB/s]
  0% 0.00/1.15M [00:00<?, ?B/s]
100% 1.15M/1.15M [00:00<00:00, 64.1MB/s]
  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 56.6MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 77.7MB/s]
  0% 0.00/685k [00:00<?, ?B/s]
100% 685k/685k [00:00<00:00, 93.6MB/s]
  0% 0.00/1.04M [00:00<?, ?B/s]
100% 1.04M/1.04M [00:00<00:00, 70.7MB/s]
  0% 0.00/238k [00:00<?, ?B/s]
100% 238k/238k [00:00<00:00, 33.8MB/s]
  0% 0.00/924k [00:00<?, ?B/s]
100% 924k/924k [00:00<00:00, 61.6MB/s]
  0% 0.00/890k [00:00<?, ?B/s]
100% 890k/890k [00:00<00:00, 59.2MB/s]
  0% 0.00/749k [00:00<?, ?B/s]
100% 749k/749k [00:00<00:00, 49.8MB/s]
  0% 0.00/938k [00:00<?, ?B/s]
100% 938k/938k [00:00<00:00, 62.0MB/s]
  0% 0.0

In [ ]:
! unzip plant-pathology-2021-fgvc8.zip -d train

Archive:  plant-pathology-2021-fgvc8.zip
replace train/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/test_images/85f8cb619c66b863.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Data processing


In [116]:
import pandas as pd
import cv2

%cd /content/gdrive/MyDrive/MLP/train/

/content/gdrive/MyDrive/MLP/train


In [117]:
df = pd.read_csv('train.csv')
print(df.head())

# df_test = pd.read_csv('sample_submission.csv')
# print(df_test.head())

                  image                           labels
0  800113bb65efe69e.jpg                          healthy
1  8002cb321f8bfcdf.jpg  scab frog_eye_leaf_spot complex
2  80070f7fb5e2ccaa.jpg                             scab
3  80077517781fb94f.jpg                             scab
4  800cbf0ff87721f8.jpg                          complex


In [118]:
def sample(df, sample_size):
  """
  This function gets an equal sample inclusive of all classes from the input dataframe
  """
  df_sampled = []
  classes = df['labels'].unique()

  for i in classes:
      g = df[df['labels'] == i].sample(sample_size)
      df_sampled.append(g)

  df_sampled = pd.concat(df_sampled)
  return df_sampled

df_sampled = sample(df, 80)
print(df_sampled)

                      image        labels
15773  eff2de8b21a83142.jpg       healthy
1169   88a9b090d7dcdbd8.jpg       healthy
13514  df38618b63c0c375.jpg       healthy
3818   9fd8a0ed6cc0b352.jpg       healthy
5723   aac590c782c7dada.jpg       healthy
...                     ...           ...
7639   b5c087bf8bb04d31.jpg  rust complex
7900   b78dc1d9dda30350.jpg  rust complex
11934  d2fb7286221e969c.jpg  rust complex
5796   aaf693d63e786009.jpg  rust complex
18454  ff331233da9091ca.jpg  rust complex

[960 rows x 2 columns]


In [119]:
def one_hot(df):
  """
  This function returns all classes and combination of classes found in the input-dataframe, and returns
  the one-hot encoded version
  """
  one_hot = pd.get_dummies(df['labels'])
  df = df.drop('labels', axis = 1)
  df = df.join(one_hot)
  return df

df_onehot = one_hot(df_sampled)
print(df_onehot)

# df_test = one_hot(df_test)
# print(df_test.head())

                      image  ...  scab frog_eye_leaf_spot complex
15773  eff2de8b21a83142.jpg  ...                                0
1169   88a9b090d7dcdbd8.jpg  ...                                0
13514  df38618b63c0c375.jpg  ...                                0
3818   9fd8a0ed6cc0b352.jpg  ...                                0
5723   aac590c782c7dada.jpg  ...                                0
...                     ...  ...                              ...
7639   b5c087bf8bb04d31.jpg  ...                                0
7900   b78dc1d9dda30350.jpg  ...                                0
11934  d2fb7286221e969c.jpg  ...                                0
5796   aaf693d63e786009.jpg  ...                                0
18454  ff331233da9091ca.jpg  ...                                0

[960 rows x 13 columns]


In [130]:
# convert dictionary into a hashable type
df_dict = df_onehot.set_index('image').T.to_dict('list')
# print(df_dict)

# gather names of all images in the image-directory
train_images = os.listdir('train_images/')

images = []
# find corresponding key to image from directory, and add that image to a list
for image in train_images:
  if image in df_dict.keys():
    img = cv2.imread('train_images/' + train_image)
    img = cv2.resize(img, (96, 96)) 
    images.append((image, img))

# sort both the list and the data equally
df_onehot = df_onehot.sort_values(by=['image'], ascending = True)
images = sorted(images)

In [133]:
print(len(train_images))
print(len(images))

6794
337


In [129]:
# replace image-name column with the corresponding images
# df_onehot.drop('image', axis = 1, inplace = True)
df_onehot['images'] = images
print(df_onehot)

ValueError: ignored